In [10]:
!pip install fastapi uvicorn kafka-python snakeviz

In [1]:
from fastapi import FastAPI
from kafka import KafkaProducer
import json
import os
import logging
import uvicorn
import asyncio
import cProfile
import pstats


logger = logging.getLogger(__name__)  # the __name__ resolve to "uicheckapp.services"

# this is a hack to multiply the work-load inserted into Kafka
names_to_multiply = os.environ.get('NAMES_TO_MULTIPLY', 'Joey,Aurelio,Evan,Donny,Foster,Dwayne,Grady,Quinton,Darin,Mickey,Hank,Kim,Peter,Jeremy,Jess,Jimmie,Vern,Pasquale,Romeo,Chris,Dale,Beau,Cliff,Timothy,Raphael,Brain,Mauro,Luke,Myron,Omar,Reynaldo,Major,Clinton,Nolan,Raymond,Lucien,Carey,Winfred,Dan,Abel,Elliott,Brent,Chuck,Dirk,Tod,Emerson,Dewey,Scot,Enrique,Al,Beatrice,Brandy,Kathy,Jane,Marcy,Shelly,Lucy,Cathy,Joanna,Doris,Lindsay,Staci,Shelia,Rosanne,Rebecca,Luz,Flora,Rosalie,Karla,Phoebe,Meagan,Virginia,Amanda,Katy,Karla,Deanne,Pearl,Christi,Victoria,Ola,Alexandra,Marina,Lorraine,Sybil,Adeline,Taylor,Anita,Aurora,Neva,Alisha,Maria,Erna,Gwendolyn,Brenda,Bethany,Sybil,Earline,June,Brandy,Sue').split(",")

_producer = None
try:
    _producer = KafkaProducer(bootstrap_servers=os.environ.get('KAFKA_BROKER', 'broker1:9093').split(","), 
                              api_version=(0, 10),
                              max_block_ms=10000)
except Exception as ex:
    logger.error('Exception while connecting Kafka')
    logger.error(str(ex))


In [2]:
app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}


saved = [False]


# insert URL into SensorLogger app: http://100.102.3.111:8080/data?device=Pixel6&person=Florin&activity=running
# https://github.com/tszheichoi/awesome-sensor-logger/#live-data-streaming
@app.post("/data")
async def data(data: dict, activity: str = None, device: str = None, person: str = None):
    if not saved[0]:
        profiler = cProfile.Profile()
        profiler.enable()
    try:
        for measurement in data["payload"]:
            timestamp_ms = int(measurement["time"]) // 1_000_000 # convert nanoseconds to milliseconds
            value_bytes = bytes(json.dumps(measurement["values"]), encoding='utf-8')
            for name in names_to_multiply + [person]:
                key_bytes = bytes(f"{name}:{device}:{activity}", encoding='utf-8')
                _producer.send(measurement['name'],
                            key=key_bytes,
                            value=value_bytes,
                            timestamp_ms=timestamp_ms) 
        _producer.flush()
        logger.info('Message published successfully.')
    except Exception as ex:
        logger.error('Exception in publishing message')
        logger.error(str(ex))

    if not saved[0]:
        profiler.disable()
        profiler.dump_stats("sensor-logger.prof")
        saved[0] = True
    
    return {"message": "Sensor data received"}


if __name__ == "__main__":
    # make available from outside by using the host 0.0.0.0
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.create_task(server.serve())

INFO:     Started server process [19157]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     172.21.0.1:38006 - "POST /data?device=Pixel6&person=Florin&activity=running HTTP/1.1" 200 OK
INFO:     172.21.0.1:38014 - "POST /data?device=Pixel6&person=Florin&activity=running HTTP/1.1" 200 OK


In [ ]:
import os
print("Open the URL: http://127.0.0.1:9001/snakeviz/%2Fhome%2Fjovyan%2Fsensor-logger.prof in your local browser. Interrupt the cell to stop the server.")
os.system('snakeviz "sensor-logger.prof" -s -p 9001 -H 0.0.0.0')

Open the URL: http://127.0.0.1:9001/snakeviz/%2Fhome%2Fjovyan%2Fsensor-logger.prof in your local browser. Interrupt the cell to stop the server.


404 GET / (172.21.0.1) 3.89ms


In [7]:
%load_ext snakeviz